In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy import stats
xr.set_options(keep_attrs=True)
import sys
sys.path.insert(1, '/glade/u/home/lettier/analysis/analysis_antarctic-asym/analysis/')
import asym_funcs as af


In [2]:
sy = 1979
ey = 1998

In [3]:
sieds = xr.open_dataset('../processed/sia_sie_daily_CDRv4_1979-2018.nc')
sieds = sieds[[f for f in sieds if 'sie' in f]]
sieds = sieds.sel(year=slice(str(sy),str(ey))).mean(dim='year')

In [4]:
wdir = '/glade/work/lettier/'

In [5]:
landmask = xr.open_dataset(wdir+'/HADISST/landmask.nc').sftlf


lats = np.linspace(-89.5,89.5,10*180) # 0.1 degree grid
lons = np.linspace(-179.5,179.5,10*360)

garea, _, _ = af.grid_area_regll(lats,lons) # compute area of these grid cells
ds = xr.DataArray(garea,dims=['latitude','longitude'],coords = {'latitude':lats, 'longitude':lons}).to_dataset(name='gridcellarea')

landmaskhighres = landmask.interp(latitude=lats)
landmaskhighres = landmaskhighres.interp(longitude=lons)

oceanarea = (ds.gridcellarea*(1.-landmaskhighres)).sum(dim='longitude')
oareanh = oceanarea.where(oceanarea.latitude>0.,drop=True)
oareash = oceanarea.where(oceanarea.latitude<0.,drop=True)

oareash['latitude'] = -oareash.latitude
oareanh = oareanh.sel(latitude=oareanh.latitude[::-1])

onhc = oareanh.cumsum(dim='latitude')
oshc = oareash.cumsum(dim='latitude')

# subtract sea ice extent from ocean area
diffnh = (onhc - 1e6*sieds[[f for f in sieds.variables if 'nh' in f]])
diffsh = (oshc - 1e6*sieds[[f for f in sieds.variables if 'sh' in f]])
diffnh = diffnh.where(diffnh>0.)
diffsh = diffsh.where(diffsh>0.)

icedge_nh = (diffnh.latitude.where(diffnh==diffnh.min(dim='latitude')).max(dim='latitude'))
icedge_sh = (diffsh.latitude.where(diffsh==diffsh.min(dim='latitude')).max(dim='latitude'))

outds = xr.merge([icedge_nh, icedge_sh])
outds.attrs['description'] = 'zonal-mean ice edge as computed from SIA/SIE obs product as listed'
outds.attrs['units'] = 'deg'

outds.to_netcdf('../processed/zonal_ice_edge_CDRv4_1979-1998clim.nc')